In [0]:
!pip install pymorphy2
!pip install pyLDAvis
import nltk
nltk.download('stopwords')

In [0]:
import gensim
import json
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
import google.colab

morph = MorphAnalyzer()

In [5]:
from google.colab import files
uploaded = files.upload()

Saving habr_texts.txt to habr_texts.txt


In [0]:
texts = open('habr_texts.txt').read().splitlines()

Нормализуем

In [0]:
stops = set(stopwords.words('russian')) | {'gt',}
def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

In [0]:

def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(top)}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stops]
        norm_texts.append(norm_words)
        
    return norm_texts

def tokenize(text):
    words = [word.strip(string.punctuation) for word in text.split()]
    words = [word for word in words if word]
    
    return words

In [0]:
texts = opt_normalize([tokenize(remove_tags(text.lower())) for text in texts], 30000)

N-граммы

In [0]:
ph = gensim.models.Phrases(texts, scoring='npmi', min_count=1, threshold=1) 
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

Словарь

In [0]:
dictionary = gensim.corpora.Dictionary(ngrammed_texts)
dictionary.filter_extremes(no_above=0.1, no_below=20)
dictionary.compactify()

In [0]:
corpus = [dictionary.doc2bow(text) for text in ngrammed_texts]

LDA

In [0]:
lda = gensim.models.LdaMulticore(corpus, 100, id2word=dictionary, eval_every=0)

In [0]:
lda.print_topics()

In [102]:
lda.log_perplexity(corpus[:10000])

-63.61912846747557

Попробую поменять параметры модели. Уменьшим число тем и добавим параметры passes, alpha

In [0]:
lda2 = gensim.models.LdaMulticore(corpus, id2word=dictionary, eval_every=1, num_topics=20, passes=3, alpha='asymmetric')

In [0]:
lda2.print_topics()

In [103]:
lda2.log_perplexity(corpus[:10000])

-10.237877543213191

Уменьшим число тем, увеличим passes

In [0]:
lda3 = gensim.models.LdaMulticore(corpus, id2word=dictionary, eval_every=1, num_topics=10, passes=5, alpha='asymmetric')

In [0]:
lda3.print_topics()

In [104]:
lda3.log_perplexity(corpus[:10000])

-9.690283925672029

Уменьшение количества тем, по-видимому, не очень хорошо сказывается на результатах

In [0]:
lda4 = gensim.models.LdaMulticore(corpus, id2word=dictionary, eval_every=1, num_topics=100, passes=5, alpha='asymmetric')

In [0]:
lda4.print_topics()

In [105]:
lda4.log_perplexity(corpus[:10000])

-11.763155684005355

Самые, на мой взгляд, убедительные темы появились в lda4. Вот некоторые из них:
(98,
  '0.112*"вакансия" + 0.026*"выборка" + 0.026*"навык" + 0.017*"москва" + 0.016*"рубль" + 0.015*"зарплата" + 0.013*"распределение" + 0.012*"профессиональный" + 0.012*"плата" + 0.011*"работодатель"')

 (94,
  '0.050*"сигнал" + 0.028*"частота" + 0.018*"звук" + 0.012*"мощность" + 0.011*"диапазон" + 0.010*"мм" + 0.009*"наушник" + 0.009*"лазер" + 0.008*"напряжение" + 0.008*"ток"'),

(5,
  '0.027*"яркость" + 0.020*"свет" + 0.019*"цвет" + 0.012*"диапазон" + 0.011*"красный" + 0.009*"координата" + 0.009*"цветок" + 0.009*"линейный" + 0.008*"цветовой" + 0.008*"кадр"')

Насколько я понимаю, хорошему качеству определения тем способствует увеличение параметров passes и num_topics. Попробуем в следующей попытке оставить passes=1

In [0]:
lda5 = gensim.models.LdaMulticore(corpus, id2word=dictionary, eval_every=1, num_topics=100, passes=1, alpha='asymmetric')

In [0]:
lda5.print_topics()

In [106]:
lda5.log_perplexity(corpus[:10000])

-63.29175312654695

Всё ещё похоже на темы, но менее убедительно. Можно сделать вывод, что параметр passes действительно влияет на результат. Например, (1,
  '0.010*"int" + 0.007*"двигатель" + 0.006*"azure" + 0.005*"you" + 0.004*"предприятие" + 0.004*"ос" + 0.003*"lt" + 0.003*"подборка" + 0.003*"суд" + 0.003*"microsoft"') - не очень похоже на тему. Теперь добавим tfidf

In [0]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)
corpus = tfidf[corpus]

In [0]:
lda4 = gensim.models.LdaMulticore(corpus, id2word=dictionary, eval_every=1, num_topics=100, passes=5, alpha='asymmetric')

In [0]:
lda4.print_topics(20)

In [114]:
lda4.log_perplexity(corpus[:10000])

-11.858708434286587

Применительно к модели, показавшей лучшие результаты, tdidf изменил их.  появились какие-то новые темы и результат скорее ухудшился. В лучшем случае 3-4 слова принадлежат какой-то общей теме. У многих слов показатель равен 0.000. Попробуем поменять параметры

In [0]:
lda2 = gensim.models.LdaMulticore(corpus, id2word=dictionary, eval_every=1, num_topics=20, passes=3, alpha='asymmetric')

In [0]:
lda2.print_topics(20)

In [115]:
lda2.log_perplexity(corpus[:10000])

-9.983256118158627

Насколько я понимаю, частотность редких слов становится еще меньше. Это не очень сильно помогает с тематическим моделированием. Видимо, нужно подобрать другие параметры.

In [0]:
lda3 = gensim.models.LdaMulticore(corpus, id2word=dictionary, eval_every=1, num_topics=10, passes=5, alpha='asymmetric')

In [113]:
lda3.print_topics()

[(0,
  '0.001*"•" + 0.001*"public" + 0.001*"игрок" + 0.001*"доклад" + 0.001*"инцидент" + 0.001*"заказчик" + 0.001*"мозг" + 0.001*"бот" + 0.001*"цифровой" + 0.001*"прибор"'),
 (1,
  '0.023*"ия" + 0.004*"удовлетворение" + 0.004*"индивидуальный" + 0.003*"void" + 0.003*"public" + 0.002*"скорректировать" + 0.002*"прогнозировать" + 0.002*"true" + 0.002*"прогноз" + 0.002*"накопить"'),
 (2,
  '0.001*"color" + 0.000*"рыба" + 0.000*"обмануть" + 0.000*"61" + 0.000*"бывший" + 0.000*"ibm" + 0.000*"тренд" + 0.000*"лидер" + 0.000*"unity" + 0.000*"доклад"'),
 (3,
  '0.000*"compute" + 0.000*"card" + 0.000*"камера" + 0.000*"сетка" + 0.000*"unity" + 0.000*"наса" + 0.000*"рейтинг" + 0.000*"ide" + 0.000*"visual" + 0.000*"studio"'),
 (4,
  '0.000*"ibm" + 0.000*"watson" + 0.000*"блокчейн" + 0.000*"датчик" + 0.000*"кандидат" + 0.000*"принтер" + 0.000*"iot" + 0.000*"наса" + 0.000*"вселенная" + 0.000*"когнитивный"'),
 (5,
  '0.000*"вселенная" + 0.000*"•" + 0.000*"инфляция" + 0.000*"облачный" + 0.000*"crm" + 0.0

Сравним некоторые полученные темы для модели с лучшими параметрами lda4 с tfidf и без него. 
Перплексия без tfidf - 11.763155684005355
Перплексия с tfidf - -11.858708434286587 
Разниц не сузественная. 
В модели с tfidf все темы поменялись кардинально (возможно, изменился их порядок, ведь мы выводим только 20 из 100). Во всех из них можно найти лишние слова. Например:
(8,
  '0.039*"уязвимость" + 0.038*"англ" + 0.033*"apk" + 0.029*"вредоносный" + 0.022*"вирус" + 0.017*"android" + 0.015*"отчёт" + 0.014*"mb" + 0.013*"sms" + 0.012*"application"'),
  Есть какие-то общие темы, например: уязвимость, вредоносный, вирус. Но другие слова как-то не очень подходят.
(98,
  '0.000*"бухгалтер" + 0.000*"1с" + 0.000*"android" + 0.000*"apk" + 0.000*"material" + 0.000*"скорректировать" + 0.000*"studio" + 0.000*"devices" + 0.000*"бекап" + 0.000*"нажимать"')
  Вроде бы есть бухгалтер и 1С, но насколько сюда подходят другие слова - сложно сказать
  (2,
  '0.007*"laravel" + 0.007*"token" + 0.007*"заказчик" + 0.006*"загрузчик" + 0.006*"import" + 0.006*"датчик" + 0.005*"конвертация" + 0.005*"гибкость" + 0.005*"аудио" + 0.005*"прямоугольник"'),
  Сложно найти много общих слов




Посмотрим на темы без tfidf

 (98,
  '0.112*"вакансия" + 0.026*"выборка" + 0.026*"навык" + 0.017*"москва" + 0.016*"рубль" + 0.015*"зарплата" + 0.013*"распределение" + 0.012*"профессиональный" + 0.012*"плата" + 0.011*"работодатель"'),
 (96,
  '0.054*"матрица" + 0.053*"вектор" + 0.033*"кандидат" + 0.027*"диаграмма" + 0.016*"плагин" + 0.015*"угол" + 0.013*"метрика" + 0.010*"трансформация" + 0.010*"луч" + 0.009*"видимость"'),

 (91,
  '0.021*"релиз" + 0.019*"скачать" + 0.017*"музыка" + 0.017*"стиль" + 0.015*"контент" + 0.014*"бот" + 0.014*"скачивание" + 0.013*"css" + 0.011*"протокол" + 0.010*"архив"'),

 (6,
  '0.037*"sql" + 0.031*"error" + 0.031*"server" + 0.022*"code" + 0.021*"case" + 0.020*"бд" + 0.017*"public" + 0.016*"was" + 0.015*"var" + 0.014*"connection"'),
 (3,
  '0.064*"температура" + 0.048*"питание" + 0.048*"корпус" + 0.043*"плата" + 0.028*"станция" + 0.022*"контроллер" + 0.022*"напряжение" + 0.018*"светодиод" + 0.018*"резистор" + 0.017*"датчик"'),
 (2,
  '0.038*"доклад" + 0.028*"конференция" + 0.011*"net" + 0.010*"выступление" + 0.009*"2017" + 0.008*"microsoft" + 0.008*"спикер" + 0.008*"обсуждение" + 0.007*"фсб" + 0.007*"джон"'),

Между словами действительно можно построить какой-то ассоциативный ряд. Такие темы составляют более значительную долю. 


NMF. Попробуем сразу изменить параметр max_features и max_df

In [0]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [0]:
stexts = [' '.join(text) for text in texts]

In [0]:
vectorizer = TfidfVectorizer(max_features=100, min_df=10, max_df=0.1, ngram_range=(1,3))
X = vectorizer.fit_transform(stexts)

In [0]:
model = NMF(n_components=30) #пусть будет количество тем - 30

In [120]:
model.fit(X)
model.components_.shape
model.transform(X).shape

(4121, 30)

In [0]:
feat_names = vectorizer.get_feature_names()

In [0]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

Что будет, если увеличить количество тем при тех же параметрах tfidf?

In [0]:
model = NMF(n_components=100)

In [0]:
model.fit(X)
model.components_.shape
model.transform(X).shape
feat_names = vectorizer.get_feature_names()

In [0]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

Вернемся к параметрам  tfidf


In [0]:
vectorizer = TfidfVectorizer(max_features=1000, min_df=100, max_df=0.1, ngram_range=(1,3))
X = vectorizer.fit_transform(stexts)

In [0]:
model = NMF(n_components=30)

In [0]:
model.fit(X)
model.components_.shape
model.transform(X).shape
feat_names = vectorizer.get_feature_names()

In [0]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

Выглядит лучше. Видимо, не стоило сильно уменьшать max_features. Попробуем снова увеличить количество тем

In [0]:
model = NMF(n_components=100)

In [0]:
model.fit(X)
model.components_.shape
model.transform(X).shape
feat_names = vectorizer.get_feature_names()

In [0]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

Некоторые темы специфичны. Возможно 100 тем это слишком много для данной модели:
конференция  2017  2017 год  международный; 00  встреча  декабрь  мероприятие

Но в целом, картина хорошая. Часто все 4 слова действительно чем-то объединены:
лекция  студент  университет  школа;
энергия  солнечный  панель  мощность;
температура  вода  воздух  прибор


Итак, посмотрим на темы лучшей NMF модели и лучшей LDA модели.
Количество тем разнится(в одной 30, а в другой 100). Возможно это не очень корректно для сравнения.  Некоторые темы вполне пересекаются. Буду выделять темы скобками. Например: 1) (94,'0.050*"сигнал" + 0.028*"частота" + 0.018*"звук" + 0.012*"мощность" + 0.011*"диапазон" + 0.010*"мм" + 0.009*"наушник" + 0.009*"лазер" + 0.008*"напряжение" + 0.008*"ток"') и ( сигнал  частота  напряжение  плата), 2) (95,'0.031*"головка" + 0.021*"температура" + 0.015*"тепло" + 0.014*"расчёт" + 0.014*"робот" + 0.014*"печать" + 0.009*"пластик" + 0.009*"принтер" + 0.008*"предпочитать" + 0.008*"совет"' )и (печать  3d  принтер  производство) 
Таким образом, можно сделать вывод, что результаты обеих моделей сопоставимы, однако в лучшей NMF меньше тем. Этот параметр опять же можно настроить лучше в обеих моделях. По сравнению с нграммной LDA NMF показала себя лучше. До этого по ошибке я рассматривал результаты LDA без нграммов и они показались лучше по качеству и сопоставимы с NMF. Ниже приведены полученные результаты по NMF и нграммной LDA. Еще ниже результаты по LDA без n-граммов. 

NMF:
0 менеджер  профессиональный  заказчик  совет
1 from  is  import  on
2 учёный  мозг  клетка  пациент
3 процессор  intel  гб  чип
4 игрок  игровой  персонаж  реальность
5 доклад  конференция  мероприятие  встреча
6 public  void  string  private
7 космический  спутник  орбита  аппарат
8 linux  microsoft  ос  studio
9 android  ios  apple  мобильный приложение
10 атака  уязвимость  злоумышленник  пароль
11 css  javascript  js  html
12 int  amp  char  const
13 облачный  облако  ibm  cloud
14 печать  3d  принтер  производство
15 дата центр  дата  вода  провайдер
16 робот  ребёнок  датчик  интеллект
17 gt  const  input  html
18 бот  чат  facebook  url
19 сигнал  частота  напряжение  плата
20 звук  музыка  слушать  волна
21 вселенная  энергия  звезда  теория
22 автомобиль  дорога  метр  город
23 камера  смартфон  аккумулятор  корпус
24 var  function  true  let
25 реклама  рубль  рекламный  доход
26 товар  скидка  покупатель  покупка
27 ip  трафик  домен  ip адрес
28 диск  резервный  хранилище  ssd
29 end  do  then  else

LDA:


[(99,
  '0.027*"хабра" + 0.023*"гипотеза" + 0.012*"linux" + 0.012*"город" + 0.011*"«вконтакте»" + 0.010*"оптимизация" + 0.010*"возраст" + 0.010*"сбор" + 0.009*"выборка" + 0.009*"r"'),
 (98,
  '0.112*"вакансия" + 0.026*"выборка" + 0.026*"навык" + 0.017*"москва" + 0.016*"рубль" + 0.015*"зарплата" + 0.013*"распределение" + 0.012*"профессиональный" + 0.012*"плата" + 0.011*"работодатель"'),
 (97,
  '0.025*"ex" + 0.016*"дизайнер" + 0.015*"офис" + 0.012*"крис" + 0.010*"unreal" + 0.009*"звук" + 0.009*"ведущий" + 0.008*"диалог" + 0.008*"glass" + 0.008*"тестовый"'),
 (96,
  '0.054*"матрица" + 0.053*"вектор" + 0.033*"кандидат" + 0.027*"диаграмма" + 0.016*"плагин" + 0.015*"угол" + 0.013*"метрика" + 0.010*"трансформация" + 0.010*"луч" + 0.009*"видимость"'),
 (94,
  '0.050*"сигнал" + 0.028*"частота" + 0.018*"звук" + 0.012*"мощность" + 0.011*"диапазон" + 0.010*"мм" + 0.009*"наушник" + 0.009*"лазер" + 0.008*"напряжение" + 0.008*"ток"'),
 (95,
  '0.031*"головка" + 0.021*"температура" + 0.015*"тепло" + 0.014*"расчёт" + 0.014*"робот" + 0.014*"печать" + 0.009*"пластик" + 0.009*"принтер" + 0.008*"предпочитать" + 0.008*"совет"'),
 (93,
  '0.046*"стикер" + 0.030*"граница" + 0.026*"цифровой" + 0.026*"выделение" + 0.025*"доска" + 0.015*"цвет" + 0.013*"цвета" + 0.012*"социальный" + 0.012*"прямоугольник" + 0.012*"фон"'),
 (92,
  '0.099*"раздел" + 0.050*"пароль" + 0.040*"папка" + 0.032*"шифрование" + 0.027*"ввести" + 0.024*"зашифровать" + 0.019*"флешек" + 0.018*"окно" + 0.018*"случайный" + 0.014*"терминал"'),
 (91,
  '0.021*"релиз" + 0.019*"скачать" + 0.017*"музыка" + 0.017*"стиль" + 0.015*"контент" + 0.014*"бот" + 0.014*"скачивание" + 0.013*"css" + 0.011*"протокол" + 0.010*"архив"'),
 (90,
  '0.101*"34" + 0.066*"28" + 0.064*"define" + 0.053*"amp;&amp" + 0.040*"output" + 0.040*"32" + 0.029*"реле" + 0.028*"high" + 0.021*"62" + 0.020*"low"'),
 (9,
  '0.065*"b" + 0.041*"int" + 0.033*"d" + 0.029*"e" + 0.027*"float" + 0.024*"long" + 0.022*"j" + 0.020*"lt" + 0.016*"n" + 0.014*"amp;&amp"'),
 (8,
  '0.037*"станция" + 0.037*"→" + 0.026*"sip" + 0.020*"ip" + 0.019*"address" + 0.019*"нажимать" + 0.018*"абонент" + 0.016*"подсеть" + 0.016*"asterisk" + 0.014*"system"'),
 (7,
  '0.074*"токен" + 0.043*"индекс" + 0.034*"подсветка" + 0.033*"иванов" + 0.031*"иван" + 0.027*"мб" + 0.026*"формирование" + 0.024*"мс" + 0.021*"предложение" + 0.020*"фраза"'),
 (6,
  '0.037*"sql" + 0.031*"error" + 0.031*"server" + 0.022*"code" + 0.021*"case" + 0.020*"бд" + 0.017*"public" + 0.016*"was" + 0.015*"var" + 0.014*"connection"'),
 (5,
  '0.027*"яркость" + 0.020*"свет" + 0.019*"цвет" + 0.012*"диапазон" + 0.011*"красный" + 0.009*"координата" + 0.009*"цветок" + 0.009*"линейный" + 0.008*"цветовой" + 0.008*"кадр"'),
 (4,
  '0.029*"uber" + 0.024*"view" + 0.023*"бизнес-логика" + 0.019*"поездка" + 0.018*"mvc" + 0.016*"product" + 0.015*"router" + 0.013*"дерево" + 0.013*"автомобиль" + 0.012*"паттерн"'),
 (3,
  '0.064*"температура" + 0.048*"питание" + 0.048*"корпус" + 0.043*"плата" + 0.028*"станция" + 0.022*"контроллер" + 0.022*"напряжение" + 0.018*"светодиод" + 0.018*"резистор" + 0.017*"датчик"'),
 (2,
  '0.038*"доклад" + 0.028*"конференция" + 0.011*"net" + 0.010*"выступление" + 0.009*"2017" + 0.008*"microsoft" + 0.008*"спикер" + 0.008*"обсуждение" + 0.007*"фсб" + 0.007*"джон"'),
 (1,
  '0.032*"java" + 0.018*"фильтр" + 0.015*"инженер" + 0.014*"советский" + 0.013*"звук" + 0.013*"производство" + 0.012*"создатель" + 0.008*"куча" + 0.008*"усилитель" + 0.008*"серия"'),
 (0,
  '0.013*"amazon" + 0.012*"loss" + 0.008*"контроллер" + 0.008*"дисплей" + 0.007*"gpu" + 0.007*"git" + 0.006*"текстура" + 0.005*"make" + 0.005*"lenovo" + 0.005*"шина"')]




LDA без n-граммов

(99,
  '0.072*"товар" + 0.047*"amazon" + 0.040*"доставка" + 0.035*"магазин" + 0.030*"покупка" + 0.024*"покупатель" + 0.019*"потребитель" + 0.019*"аккаунт" + 0.014*"ветка" + 0.012*"продавец"'),
 (98,
  '0.015*"окно" + 0.013*"меню" + 0.012*"панель" + 0.010*"xml" + 0.009*"редактор" + 0.007*"1с" + 0.007*"график" + 0.007*"диаграмма" + 0.007*"анимация" + 0.007*"watson"'),
 (97,
  '0.052*"клетка" + 0.022*"ген" + 0.020*"образование" + 0.019*"ребёнок" + 0.018*"тело" + 0.015*"женщина" + 0.014*"болезнь" + 0.013*"заболевание" + 0.011*"организм" + 0.011*"пациент"'),
 (96,
  '0.014*"is" + 0.014*"system" + 0.009*"on" + 0.009*"boot" + 0.009*"show" + 0.009*"version" + 0.009*"laravel" + 0.008*"master" + 0.008*"use" + 0.008*"device"'),
 (95,
  '0.077*"батарея" + 0.074*"tesla" + 0.067*"аккумулятор" + 0.052*"ёмкость" + 0.036*"ячейка" + 0.031*"производство" + 0.028*"model" + 0.028*"охлаждение" + 0.018*"напряжение" + 0.016*"заявить"'),
 (93,
  '0.053*"китай" + 0.038*"цифровой" + 0.031*"правительство" + 0.029*"китайский" + 0.024*"блокировка" + 0.024*"провайдер" + 0.021*"социальный" + 0.021*"vpn" + 0.015*"корпоративный" + 0.013*"зарубежный"'),
 (94,
  '0.045*"статический" + 0.039*"android" + 0.033*"динамический" + 0.030*"иконка" + 0.025*"короткий" + 0.016*"удалять" + 0.015*"•" + 0.015*"отображаться" + 0.014*"длинный" + 0.014*"удаление"'),
 (92,
  '0.020*"камера" + 0.018*"смартфон" + 0.008*"гб" + 0.008*"ноутбук" + 0.007*"дисплей" + 0.007*"разрешение" + 0.006*"процессор" + 0.006*"корпус" + 0.006*"видеокарта" + 0.005*"рубль"'),
 (91,
  '0.044*"космический" + 0.040*"земля" + 0.036*"орбита" + 0.034*"аппарат" + 0.033*"спутник" + 0.028*"полёт" + 0.026*"луна" + 0.025*"марс" + 0.019*"наса" + 0.018*"ракета"'),
 (90,
  '0.032*"диск" + 0.023*"ssd" + 0.019*"накопитель" + 0.018*"скидка" + 0.015*"драйвер" + 0.015*"блокчейн" + 0.012*"операционный" + 0.012*"финансовый" + 0.009*"файловый" + 0.009*"платёж"'),
 (9,
  '0.008*"тестовый" + 0.007*"crm" + 0.007*"менеджер" + 0.006*"функционал" + 0.006*"планирование" + 0.006*"внедрение" + 0.006*"автоматизация" + 0.006*"git" + 0.005*"баг" + 0.004*"релиз"'),
 (8,
  '0.046*"контейнер" + 0.023*"сборка" + 0.020*"docker" + 0.017*"консоль" + 0.016*"linux" + 0.015*"run" + 0.012*"nginx" + 0.012*"d" + 0.012*"каталог" + 0.010*"папка"'),
 (7,
  '0.093*"ms" + 0.047*"sec" + 0.028*"00" + 0.026*"sum" + 0.023*"oracle" + 0.018*"person" + 0.013*"sap" + 0.012*"bytes" + 0.010*"акция" + 0.010*"handle"'),
 (6,
  '0.037*"двигатель" + 0.025*"квеста" + 0.023*"загадка" + 0.018*"город" + 0.017*"зона" + 0.016*"производство" + 0.015*"партнёр" + 0.012*"электроника" + 0.010*"плата" + 0.010*"пуск"'),
 (5,
  '0.028*"раздел" + 0.022*"цифровой" + 0.014*"токен" + 0.010*"распространение" + 0.009*"свободный" + 0.007*"зашифровать" + 0.007*"ввести" + 0.007*"иван" + 0.007*"иванов" + 0.007*"предметный"'),
 (4,
  '0.037*"err" + 0.034*"функциональный" + 0.033*"var" + 0.028*"function" + 0.028*"error" + 0.026*"const" + 0.023*"rate" + 0.023*"res" + 0.022*"javascript" + 0.020*"user"'),
 (3,
  '0.012*"сигнал" + 0.011*"протокол" + 0.009*"трафик" + 0.008*"частота" + 0.007*"маршрут" + 0.007*"сетевой" + 0.006*"маршрутизатор" + 0.006*"соединение" + 0.004*"сессия" + 0.003*"tcp"'),
 (2,
  '0.034*"linux" + 0.018*"http" + 0.017*"оптимизация" + 0.017*"серверный" + 0.016*"балансировка" + 0.014*"кэширование" + 0.013*"сжатие" + 0.013*"проксить" + 0.013*"доход" + 0.013*"налог"'),
 (1,
  '0.023*"нативный" + 0.023*"контент" + 0.019*"веб" + 0.015*"facebook" + 0.014*"ios" + 0.013*"бот" + 0.012*"мессенджер" + 0.011*"социальный" + 0.010*"операционный" + 0.010*"предпочитать"'),
 (0,
  '0.023*"массив" + 0.023*"b" + 0.020*"const" + 0.018*"int" + 0.017*"n" + 0.014*"y" + 0.013*"lt" + 0.011*"p" + 0.010*"указатель" + 0.009*"json"')